In [90]:
import urllib.request
import re
from bs4 import BeautifulSoup
from lxml import html
import pandas as pd
# import xmltojson
import json
import requests

In [91]:
def range_price(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
        
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "lxml")
    lowend = soup.find_all("span", {"class":"value", "itemprop":"lowprice", "content": True})
    highend = soup.find_all("span", {"class":"value", "itemprop":"highprice", "content": True})
    quantityList = []
    rangePrice = ""

    for i in lowend:
        rangePrice+=(i["content"])
        rangePrice+=" - "
        break

    for i in highend:
        rangePrice+=(i["content"])
        break

    return rangePrice


In [92]:
def scrape_handguns_helper(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
    url+= "=undefined&start=0&sz=100"
        
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "lxml")
    ammo = soup.find_all("div", class_="pdp-link")
    ammoPrice = soup.find_all("div",  class_="tile-body")
    names = []
    prices = []
    actual_prices = []
    for i in ammo:
        names.append(i.find("span").text)

    for i in ammoPrice:
        # if the product has a range of prices go here
        if '<span class="range">' in str(i):
            x = i.find("a", {"class": "link", "href":True})
            prices.append(x["href"])

        # if the product has a slashed price, go here
        else:
            x = (i.find("span", class_="sales"))
            prices.append(x.find("span", {"class":"value", "content": True}))


    # add prices in the prices list
    for i in prices:
        if "http" in i:
            rp = range_price(i)
            actual_prices.append(rp)

        else:
            htmlPriceTag = str(i)
            splitHtmlPriceTag = htmlPriceTag.split(" ")
            p =  splitHtmlPriceTag[2].strip('content="')
            actual_prices.append(str(p))

   
    try:
        productList = pd.DataFrame(
            {"name": names, "price": actual_prices}
        )
        return productList

    except:
        print("something went wrong")

In [93]:
# https://www.cheaperthandirt.com/9mm-luger-parabellum/ammunition/handgun-ammo/?https://www.cheaperthandirt.com/9mm-luger-parabellum/ammunition/handgun-ammo/=undefined&start=0&sz=36
handgunByCaliberLink = "https://www.cheaperthandirt.com/shop-by?cgid=78&searchBy=Caliber"
def scrape_handguns(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
        
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "lxml")
    sections = soup.find_all("ul",class_= "col-lg-11 right-list")

    # get caliber number set
    caliberNumbers = []
    x = soup.find_all('div', class_="refinements")
    for i in x:
        cn = i.find_all("div", {"id": True})
        for j in cn:
            caliberNumbers.append((j["id"]))

            
    # for i in x:
    #     cn = i.find_all("div", {"id": True})
    #     for j in cn:
    #         print(j)
    #         print("_______________________________\n")

    dirtyLinks = []  
    listofDF1s = []
    
    for i in sections:
        dirtyLinks.append(i.find_all("a"))

    caliberNumberDF = pd.DataFrame(
        {"number": caliberNumbers, "section": dirtyLinks}
    )
    for i in caliberNumberDF["section"]:
        calibers = []  # names
        linkToCaliberList = []  # links
        for j in i:
            aLink = re.findall("(?P<url>https?://[^\s]+)", str(j))
            linkToCaliberList.append(str(aLink[0]).replace('">', ""))
            calibers.append(j.find("span").text)
        sectionsDF = pd.DataFrame(
            {"caliberType": calibers, "links": linkToCaliberList}
        )
        listofDF1s.append(sectionsDF)

    c = []
    aDF = []
    for i in listofDF1s:
        calibers = []
        df = []
        for k in i["links"]:
            df.append(scrape_handguns_helper(k))
        for j in (i["caliberType"]):
            calibers.append(j)
        c.append(calibers)
        aDF.append(df)

    caliberNumberDF = pd.DataFrame({"caliber_number":caliberNumbers, "caliber_type":c, "dfForCaliber":aDF})
    for i in caliberNumberDF["dfForCaliber"]:
        print(i)
        print("_____________________________________________________________")
        


    # # get links and calibers
    # for i in dirtyLinks:
    #     for j in i:
    #         aLink = re.findall("(?P<url>https?://[^\s]+)", str(j))
    #         linkToCaliberList.append(str(aLink[0]).replace('">', ""))
    #         calibers.append(j.find("span").text)

    # for i in dirtyLinks:
    #     print(i)
    #     print("_____________________________________________________________")
        
    # handgunCaliberLinks = pd.DataFrame(
    #     {"caliber": calibers, "links": linkToCaliberList}
    # )

    # productList = []
    # for l in handgunCaliberLinks["links"]:            # seperate by link?
    #     productList.append(scrape_handguns_helper(l))

    # return productList

In [94]:
handgunDF = (scrape_handguns(handgunByCaliberLink))

[                                                name   price
0  American Quality 10mm Auto Ammunition 250 Bulk...  199.89
1  Browning X-Point Defense 10mm Auto Ammunition ...   29.89
2  Sellier & Bellot 10mm Auto Ammunition 50 Round...   49.89
3  Magtech 10mm Auto Ammunition 50 Rounds FMJ 180...   49.89
4  Buffalo Bore Tactical 10mm Auto Ammunition 20 ...   39.89
5  CCI Blazer 10mm Auto Ammunition 50 Rounds 200 ...   59.89
6  Buffalo Bore Tactical 10mm Auto Ammunition 20 ...   34.89]
[                                                name  price
0  Winchester USA .25 ACP Ammunition 50 Rounds, F...  49.89]
[                                                name  price
0  Buffalo Bore .327 Federal Ammunition 20 Rounds...  34.89,                                                 name  price
0  Buffalo Bore .32 H&R Magnum +P Ammunition 20 R...  34.89
1  Buffalo Bore .32 H&R Magnum +P Ammunition 20 R...  34.89,                                                 name   price
0  American Quality .357

In [95]:
from pathlib import Path
import os, shutil, glob
rel = "jsonFiles/cheaperThanDirt/ammo/handgun/"
cwd = Path.cwd()
currentPath = str(Path.cwd())+""
aCount = 0
savePath = str(cwd).replace("pyScripts", rel)

print(savePath)
for i in handgunDF:
    result = i.to_json(orient="split")
    parsed = json.loads(result)
    data = json.dumps(parsed, indent=4)
    with open(f"{savePath}{aCount}.json", "w") as fp:
        json.dump(data, fp)
    aCount += 1


# 

/Users/chakaneshegog/Desktop/crispy-computing-machine/jsonFiles/cheaperThanDirt/ammo/handgun/


TypeError: 'NoneType' object is not iterable